In [ ]:
import yaml
from omegaconf import OmegaConf
from yolo_ev.module.model_module import ModelModule
from yolo_ev.module.data_module import DataModule

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import loggers as pl_loggers


save_dir = './result'

yaml_file = "./config/param.yaml"
with open(yaml_file, 'r') as file:
    config = yaml.safe_load(file)
config = OmegaConf.create(config)

data = DataModule(config)
model = ModelModule(config)



In [ ]:
import torch

ckpt = torch.load("./weight/yolox_m.pth")
model.model.load_state_dict(ckpt['model'])

In [ ]:
import yaml
from omegaconf import OmegaConf
from yolo_ev.module.data.dataset.dsec_det.dsec_det_dataset import DsecDetDataset
from yolo_ev.module.data.dataset.dsec_det.build_dsec_det_dataset import build_coco_dataset
yaml_file = "./config/param_dsec.yaml"
with open(yaml_file, 'r') as file:
    config = yaml.safe_load(file)
config = OmegaConf.create(config)

dataset = build_coco_dataset(config, mode="train")

In [ ]:
dataset.subsequence_directories

In [ ]:
dataset[0]